<a href="https://colab.research.google.com/github/srishmaalladi/Katha-vachak/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
!pip install pyngrok
!pip install flask_cors
!pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"



  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.1
    Uninstalling tokenizers-0.20.1:
      Successfully uninstalled tokenizers-0.20.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.0.dev0
    Uninstalling transformers-4.46.0.dev0:
      Successfully uninstalled transformers-4.46.0.dev0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-mg9r8g9s
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-mg9r8g9s
  Resolved https://github.com/huggingface/transformers.git to commit 816f44249

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from unsloth import FastLanguageModel
model,tokenizer=FastLanguageModel.from_pretrained('/content/drive/MyDrive/final_model')


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.46.0.dev0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from unsloth import FastLanguageModel
from transformers import TextStreamer, AutoTokenizer, AutoModelForCausalLM
import torch
from flask_cors import CORS


# Initialize Flask app
app = Flask(__name__)

# Set up ngrok for public access
ngrok.set_auth_token("2nTaRxHGdDYre0DMbc02Do1TzPz_5dYfMsAZ8bfMidRijWmY7")
public_url = ngrok.connect(5000).public_url

# Print the public URL for external access
print(f"To access the global link, please click {public_url}")

# Load the model and tokenizer
story_keywords = ["story", "generate", "tale", "narrative", "adventure"]

def contains_story_keywords(user_input):
    """Check for any story-related keywords in the user input"""
    return any(keyword in user_input.lower() for keyword in story_keywords)


@app.route("/", methods=["POST"])
def generate_story():
    user_input = request.json.get("input", "")  # Get user input from the POST request
    # Check if the input contains story-related keywords
    if contains_story_keywords(user_input):
        # Prepare inputs for the model (direct input from the user)
        inputs = tokenizer([user_input], return_tensors="pt").to("cuda")

        # Enable native 2x faster inference using FastLanguageModel
        FastLanguageModel.for_inference(model)  # This line is essential for performance

        # Generate the story
        generated_story = model.generate(**inputs)

        # Decode the generated tokens to text
        story_text = tokenizer.decode(generated_story[0], skip_special_tokens=True)

        # Return the generated story as a JSON response
        return jsonify({"story": story_text})

    else:
        # User input does not contain story-related keywords; provide an alternative response
        return jsonify({"message": "I'm trained to generate stories. Your input doesn't seem to ask for one."})

# Run the Flask app
if __name__ == "__main__":
    app.run(port=5000)


To access the global link, please click https://bbd4-34-126-181-196.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-7-c06a0547cc21>", line 39, in generate_story
    generated_story = model.generate(**inputs)
  File "/usr/local/lib/python3.10/dist